In [1]:
%load_ext autoreload
%autoreload 2

import numpy  as np
import pandas as pd
import h5py

from tqdm import tqdm
from astropy import units as u
from scipy.interpolate import interp1d

import ehtplot
from matplotlib import pyplot as plt, cm

from common import dalt
from common import hallmark as hm
from common import viz
from common import io_ipole as io

In [2]:
pf_img = hm.ParaFrame('model/Illinois_thermal/{freq}/{mag}a{aspin:g}_w{win:d}/img_s{snapshot:d}_Rh{Rhigh:d}_i{inc:d}.h5')

for k in set(pf_img.keys()) - {'path'}:
    globals()[k] = np.unique(pf_img[k])
    print(k, globals()[k][:16])

win [5]
snapshot [5000 5001 5002 5003 5004 5005 5006 5007 5008 5009 5010 5011 5012 5013
 5014 5015]
aspin [0.5  0.94]
Rhigh [10 40]
inc [50 70]
freq ['230GHz']
mag ['M' 'S']


In [3]:
pf_summ = hm.ParaFrame('cache/Illinois_thermal_w{win:d}/{mag}a{aspin:g}_i{inc:d}/summ_Rh{Rhigh:d}_{freq}.tsv')

for k in set(pf_summ.keys()) - {'path'}:
    globals()[k] = np.unique(pf_summ[k])
    print(k, globals()[k][:16])

win [3 4 5]
inc [ 10  30  50  70  90 110 130 150 170]
Rhigh [  1  10  40 160]
aspin [-0.94 -0.5   0.    0.5   0.94]
freq ['230GHz' '2um' '86GHz']
mag ['M' 'S']


In [4]:
pf_sed = hm.ParaFrame('cache/Illinois_thermal_w{win:d}/{mag}a{aspin:g}_i{inc:d}/sed_Rh{Rhigh:d}.h5')

for k in set(pf_sed.keys()) - {'path'}:
    globals()[k] = np.unique(pf_sed[k])
    print(k, globals()[k][:16])
    
def readsed(f, snapshot=None):

    with h5py.File(f) as h:
        time = h['time'][:]
        nu   = h['nu'  ][:]
        knd  = h['knd' ][:]
        avg  = h['avg' ][:]
      # err  = h['err' ][:]
        rlz  = h['len' ][:]

    if not all(rlz == 16):
        print('WARNING: less than 16 realizations:', f)

    nuLnu = interp1d(time, avg, axis=0)
    
    return time, nu, nuLnu, [k.decode("utf-8") for k in knd]

win [3 4 5]
inc [ 10  30  50  70  90 110 130 150 170]
Rhigh [  1  10  40 160]
aspin [-0.94 -0.5   0.    0.5   0.94]
mag ['M' 'S']


In [5]:
def Fnu_to_nuLnu(nu, Fnu):
    d = 8.127e3 * u.pc
    S = 4 * np.pi * d * d
    return (Fnu*u.Jy * S * nu*u.Hz).to(u.erg/u.second).value

In [6]:
def plot_snapshot(a, m, i, Rh, w):
    
    sel_sed = pf_sed(aspin=a)(mag=m)(inc=i)(Rhigh=Rh)(win=w)
    assert len(sel_sed) == 1
    
    sel_summ = pf_summ(aspin=a)(mag=m)(inc=i)(Rhigh=Rh)(win=w)
    assert len(sel_summ) == 3

    sel_img = pf_img(aspin=a)(mag=m)(inc=i)(Rhigh=Rh)(win=w)
    assert len(sel_img) == 1000 # 3000
    
    time, nu, nuLnu, knd = readsed(sel_sed.path.iloc[0])
    df_86GHz   = pd.read_csv(sel_summ(freq='86GHz' ).path.iloc[0], sep='\t')
    df_230GHz  = pd.read_csv(sel_summ(freq='230GHz').path.iloc[0], sep='\t')
    df_2um     = pd.read_csv(sel_summ(freq='2um'   ).path.iloc[0], sep='\t')
    mov_230GHz = io.load_mov(tqdm(sel_img(freq='230GHz').path.iloc[:2]))
    
    for s, t in enumerate(tqdm(mov_230GHz.meta.time)):
        hr = df_230GHz.time_hr.iloc[s] - df_230GHz.time_hr.iloc[0]
        
        fig = plt.figure(constrained_layout=True, figsize=(20,8))
        
        gs  = fig.add_gridspec(4,5)
        ax0 = fig.add_subplot(gs[0:2,0])
        ax1 = fig.add_subplot(gs[0:2,1])
        ax2 = fig.add_subplot(gs[0:2,2])
        ax3 = fig.add_subplot(gs[:,3:])
        ax4 = fig.add_subplot(gs[2,:3])
        ax5 = fig.add_subplot(gs[3,:3])

        r = df_86GHz.iloc[s]
        viz.show(mov_86GHz, s=i, ax=ax1, cmap='afmhot_10us', vmin=0, vmax=df_86GHz.Imax.median())
        viz.ellipse(r.alpha0, r.beta0, r.major_FWHM, r.minor_FWHM, r.PA, ax=ax1, color='w')

        r = df_230GHz.iloc[s]
        viz.show(mov_230GHz, s=s, ax=ax1, cmap='afmhot_10us', vmin=0, vmax=df_230GHz.Imax.median())
        viz.ellipse(r.alpha0, r.beta0, r.major_FWHM, r.minor_FWHM, r.PA, ax=ax1, color='w')

        r = df_2um.iloc[s]
        viz.show(mov_2um, s=i, ax=ax1, cmap='afmhot_10us', vmin=0, vmax=df_86GHz.Imax.median())
        viz.ellipse(r.alpha0, r.beta0, r.major_FWHM, r.minor_FWHM, r.PA, ax=ax1, color='w')

        viz.step(ax3, nu, nuLnu(t), label=knd)
        ax3.errorbar(
            [86e9, 230e9, 1.4141e+14, 1.45e18], 
            [Fnu_to_nuLnu(86e9,1.9), Fnu_to_nuLnu(230e9,2.4), Fnu_to_nuLnu(1.4141e+14,1e-3), 1e33],
            yerr=[Fnu_to_nuLnu(86e9,0.2), 0, Fnu_to_nuLnu(1.4141e+14,1e-3)/2, 1e33/2],
            uplims=[False,False,True,True],
            fmt='o', color='k'
        )
        ax3.set_xlim(1e9,  1e23)
        ax3.set_ylim(1e27, 1e37)
        ax3.set_xlabel(r'$\nu$ [Hz]')
        ax3.set_ylabel(r'$\nu L_\nu$ [erg/s]')
        ax3.text(2e9, 4e36, f'Time = {t:g} = {hr:g} hr', fontsize=16)
        ax3.legend(loc='upper right')
        
        p = ax4.plot(df_86GHz .time, df_86GHz .Ftot, label='86GHz flux')
        ax4.axhline(y=1.9, linewidth=1, alpha=0.5, color=p[0].get_color())
        p = ax4.plot(df_230GHz.time, df_230GHz.Ftot, label='230GHz flux')
        ax4.axhline(y=2.4, linewidth=1, alpha=0.5, color=p[0].get_color())
        p = ax4.plot(df_2um   .time, df_2um   .Ftot, label='2um flux')
        ax4.axhline(y=1e-3,linewidth=1, alpha=0.5, color=p[0].get_color())
        ax4.set_xlabel('Time [M]')
        ax4.set_ylabel(r'$F_\nu$ [Jy]')
        ax4.legend(loc='upper right')
        ax4.axvline(x=t.value, color='k')
        ax4.set_yscale('log')
        
        p = ax5.plot(df_86GHz .time, df_86GHz .major_FWHM, alpha=0.5, label='86GHz major')
        ax5.axhline(y=120, linewidth=1, alpha=0.5, color=p[0].get_color())
        p = ax5.plot(df_230GHz.time, df_230GHz.major_FWHM, alpha=0.5, label='230GHz major')
        ax5.axhline(y=95,  linewidth=1, alpha=0.5, color=p[0].get_color())
        p = ax5.plot(df_230GHz.time, df_230GHz.minor_FWHM, alpha=0.5, label='230GHz minor')
        ax5.axhline(y=38,  linewidth=1, alpha=0.5, color=p[0].get_color())
        ax5.set_xlabel('Time [M]')
        ax5.set_ylabel(r'Angular size [$\mu$as]')
        ax5.legend(loc='upper right')
        ax5.axvline(x=t.value, color='k')
        
        fig.savefig(f'output/movie/{m}a{a}_i{i}_Rh{Rh}_w{w}_s{s:04d}.png')
        
        plt.close(fig)

In [13]:
plot_snapshot(0.5, 'M', 50, 10, 5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.29it/s]
